In [13]:
using LinearAlgebra
using FrankWolfe
include("terms_and_polynomials.jl")
include("auxiliary_functions.jl")
include("objective_functions.jl");

# OAVI Algorithm

In [26]:
"""
Creates OAVI feature transformation fitted to X_train

# Arguments
- 'X_train::Vector{Vector{Float64}}': training data
- 'max_degree::Int64': max degree of polynomials computed (default 10)
- 'psi::Float64': vanishing extent (default 0.1)
- 'epsilon::Float64': accuracy for convex optimizer (default 0.001)
- 'tau::Union{Float64, Int64}': upper bound on norm of coefficient vector

# Returns
- 'X_train_transformed::Vector{Vector{Float64}}': transformed X_train
- 'sets::sets_avi': instance of mutable struct keeping track of sets for AVI 
"""  # line 14
function fit(X_train::Vector{Vector{Float64}}; 
        max_degree::Int64=10, psi::Float64=0.1, epsilon::Float64=0.001, tau::Union{Float64, Int64}=1000,
        lmbda::Float64=0., tol::Float64=0.0001, objective_type::String="L2Loss", region_type::String="L1Ball", 
        oracle_type::String="CG", max_iters::Int64=10000, inverse_hessian_boosting::String="false")

    m, n = length(X_train), length(X_train[1])
    
    sets = SetsOandG([nothing], [nothing], [nothing], [nothing], [nothing],
    zeros(Int64, length(X_train), 1), ones(Float64, length(X_train), 1), [],
    [nothing], zeros(Float64, m, 0), 
    nothing)
    
    degree = 0
    while degree < max_degree
        degree += 1 
        # line 30
        border_terms_raw, border_evaluations_raw, non_purging_indices = construct_border(sets.O_terms, sets.O_evaluations, X_train)
        border_terms = border_terms_raw[:, non_purging_indices]
        border_evaluations = border_evaluations_raw[:, non_purging_indices]
        
        O_indices = []
        leading_terms = []
        G_coefficient_vectors = nothing

        data = sets.O_evaluations
        data_squared = data' * data
        data_squared_inverse = nothing

        for col_idx in 1:size(border_terms, 2)

            if G_coefficient_vectors != nothing
                G_coefficient_vectors = vcat(G_coefficient_vectors, zeros(Float64, 1, size(G_coefficient_vectors, 2)))
            end

            term_evaluated = border_evaluations[:, col_idx] 
            data_term_evaluated = data' * term_evaluated
            term_evaluated_squared = term_evaluated' * term_evaluated
            
            f, grad!, region = nothing, nothing, nothing           
            
            if objective_type == "L2Loss"
                Loss, f, grad! = construct_L2Loss(data, term_evaluated; lmbda=lmbda, data_squared=data_squared, labels_squared=term_evaluated_squared, 
                                    data_squared_inverse=data_squared_inverse, data_labels=data_term_evaluated)
            end

            if region_type == "L1Ball"
                region = FrankWolfe.LpNormLMO{1}(tau-1)
            end

            @assert f != nothing "Objective function f not defined."
            @assert grad! != nothing "Gradient of f not defined."
            @assert region != nothing "Feasible region not defined."

            # compute initial point 
            x0 = Vector(compute_extreme_point(region, zeros(Float64, size(data, 2))))

            coefficient_vector = call_oracle(f, grad!, region, x0)
            coefficient_vector = hcat(coefficient_vector, [1])
            coefficient_vector = reshape(coefficient_vector, size(data, 2)+1, 1)
            data_with_labels = hcat(data, term_evaluated)
            loss = (1 / size(data, 1)) * norm(data_with_labels * coefficient_vector, 2)^2
            
            if loss <= psi
                leading_terms = append!(leading_terms, col_idx)
                G_coefficient_vector = update_coefficient_vectors(G_coefficient_vectors, coefficient_vector)
            else
                O_indices = append(O_indices, col_idx)
                data, data_squared, data_squared_inverse = streaming_matrix_updates(data, data_squared, data_term_evaluated,
                                                            term_evaluated, term_evaluated_squared, data_squared_inverse)
            end
            
        end
        
        update_leading_terms(sets, leading_terms)
        update_G(sets, G_coefficient_vectors)
        
        if O_indices == []
            break
        else
            update_O(sets, border_terms[:, O_indices], border_evaluations[:, O_indices], O_indices)
        end
    end
    
    X_train_transformed = sets.G_evaluations
    
    if size(X_train_transformed, 2) != 0
        X_train_transformed = abs.(X_train_transformed)
    else
        X_train_transformed = nothing
    end
    
    return X_train_transformed, sets
    
end

fit

In [27]:
a, b, c = fit([[1.0, 0.1], 
[1.0, .7], 
[0.567, 0.931], 
[0.123, 0.567]])

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

...

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ F

Excessive output truncated after 524339 bytes.┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clipping. Convergence might be not guaranteed.
└ @ FrankWolfe C:\Users\pusty\.julia\packages\FrankWolfe\MDe7s\src\linesearch.jl:367
┌ Warning: Smoothness estimate run away -> hard clippi

LoadError: UndefVarError: `append` not defined

In [7]:
function update_coefficient_vectors(G_coefficient_vectors, coefficient_vector; first=false)
    if G_coefficient_vectors == nothing
        G_coefficient_vectors = coefficient_vector
    else
        if first
            lt_indices = find_first_non_zero_entries(G_coefficient_vectors)
        else
            lt_indices = find_last_non_zero_entries(G_coefficient_vectors)
        end
        
        removable_set = Set(lt_indices)
        
        indices = [x for x in 1:size(G_coefficient_vectors, 1) if x ∉ removable_set]
        
        if length(indices) == size(coefficient_vector, 1)
            updated_coefficient_vector = zeros(size(G_coefficient_vectors, 1), 1)
            updated_coefficient_vectors[indices, :] = coefficient_vector
            G_coefficient_vectors = hcat(G_coefficient_vectors, updated_coefficient_vector)
        end
    end
    return G_coefficient_vectors
end

update_coefficient_vectors (generic function with 1 method)

In [26]:
function streaming_matrix_updates(A, A_squared, A_a, a, a_squared; A_squared_inv=nothing, built_in::Bool=false)
    B_squared_inv = nothing

    if built_in
        B = hcat(A, a)
        B_squared = B' * B
        if A_squared_inv != nothing
            B_squared_inv = inv(B_squared)
        end
    else
        B = hcat(A, a)

        b = A_a

        B_squared = hcat(A_squared, b)
        B_squared = vcat(B_squared, vcat(b, a_squared)')

        if A_squared_inv != nothing 
            # write B_squared_inv as S = | S_1, s_2|
            #                            | s_2.T s_3|

            A_squared_inv_b = A_squared_inv * b
            b_A_squared_inv_b = (b' * A_squared_inv_b)[1]

            s_2 = A_squared_inv + ((A_squared_inv_b * A_squared_inv_b') ./ (a_squared - b_A_squared_inv_b))
            s_2 = (s_2 * b) ./ a_squared

            s_3 = (1 - (b' * s_2)[1]) / a_squared

            S_1 = A_squared_inv - (A_squared_inv_b * s_2')

            B_squared_inv = hcat(S_1, s_2)
            B_squared_inv = vcat(B_squared_inv, vcat(s_2, s_3)')            
        end
    end

    return B, B_squared, B_squared_inv
    
end

streaming_matrix_updates (generic function with 2 methods)

In [10]:
using Random

In [63]:
dim = 100000
A = randn(dim, 500)
A_squared = A' * A
A_squared_inv = copy(A_squared)
A_squared_inv = inv(A_squared_inv)

a = randn(dim, 1)
a_squared = (a' * a)[1]

A_a = A' * a

B, B_2, B_2_1 = streaming_matrix_updates(A, A_squared, A_a, a, a_squared; A_squared_inv=A_squared_inv, built_in=false)

C = hcat(A, a)

C_2 = C' * C

C_2_1 = copy(C_2)
C_2_1 = inv(C_2_1)

display(norm(C_2 .- B_2, 2))
norm(C_2_1 .- B_2_1, 2)

6.701740245802964e-13

2.044444670267016e-6

In [118]:
a = randn(3, 1)
a' * a

1×1 Matrix{Float64}:
 3.6904045402212504

In [ ]:
# streaming matrix updates anschauen!